In [1]:
import segar

In [21]:
from typing import Optional, Tuple

from gym.spaces import Discrete
from IPython.display import Image
from matplotlib.pyplot import imshow
import numpy as np
import PIL

from segar.rules.transitions import TransitionFunction, Aggregate, SetFactor, Differential
from segar.factors import (
    Position,
    Velocity,
    Acceleration,
    Size,
    Mass,
    Shape,
    Circle,
    NumericFactor,
    FACTORS,
    Force,
    Mobile,
    Label,
    Text
)
from segar.mdps.initializations import Initialization
from segar.mdps.mdps import MDP
from segar.mdps.observations import ObjectStateObservation
from segar.mdps.tasks import Task
from segar.parameters import Gravity
from segar.rendering.rgb_rendering import RGBRenderer, register_rule
from segar.sim import change_precision, Simulator
from segar.tools.sample_trajectories import rollout_sim_only, rollout, save_gif
from segar.things import Entity, Object, Thing, Ball
from segar.things import SquareWall


change_precision(0.01) # Because changing the size makes collision checks a bit less precise

In [22]:
class Angle(NumericFactor[float], default=0., range=[-float(np.pi) / 2., float(np.pi) / 2.]):
    pass


class AngularVelocity(NumericFactor[float], default=0., range=[-float(np.pi), float(np.pi)]):
    pass


class AngularAcceleration(NumericFactor[float], default=0.):
    pass


class PoleMassProp(Mass, range=[0., 1.], default=0.1):
    pass


class PoleLength(NumericFactor[float], default=1.):
    pass


first_factors = [ft for ft in FACTORS if ft not in (Velocity, AngularVelocity, Position, Angle, Force)]
sim = Simulator(factor_update_order=(first_factors[:], [Force, Velocity, AngularVelocity], [Angle]))

_mountaincar_range = [-1.2, .5]
_mountaincar_length = _mountaincar_range[1] - _mountaincar_range[0]
_mountaincar_center = (_mountaincar_range[0] + _mountaincar_range[1]) / 2.

_arena_range = sim.boundaries
_arena_length = _arena_range[1] - _arena_range[0]
_arena_center = (_arena_range[0] + _arena_range[1]) / 2.


def change_basis(x, buffer=0.1):
    # center and rescale
    x -= _arena_center
    # We want a buffer so we don't have to deal with object size.
    x *= (_mountaincar_length) / (_arena_length - 2. * buffer)
    # shift
    x += _mountaincar_center
    return x
    
    
def change_back(x, buffer=0.1):
    x -= _mountaincar_center
    x /= (_mountaincar_length) / (_arena_length - 2. * buffer)
    x += _arena_center
    return x


@TransitionFunction
def accelerate_like_mountian_car(o_factors: Tuple[Size, Position, Acceleration, Mass], gravity: Gravity) -> Tuple[Aggregate[Acceleration], Differential[Size]]:
    
    size, pos, acc, mass = o_factors
    pos_x = pos[0]
    acc_x = acc[0]
    # mountaincar area is just the x axis minus the size / 2 of the object (hack because assuming circle shape). But the mountaincar range is defined by -1.2 to .5 and our arena is -1. to 1., so change basis
    pos_x = change_basis(pos_x)
    slope = np.cos(3 * pos_x)
    da_x = change_back(-gravity * mass * slope)
    
    da = Acceleration([da_x, 0.])
    
    # To simulate visual change depending on where on the curve the ball is
    d_size = 0.#0.2 * slope * vel_x
    return (Aggregate[Acceleration](acc, da), Differential[Size](size, d_size))


@TransitionFunction
def mountaincar_wall_collision(obj: Object, wall: SquareWall) -> Tuple[SetFactor[Velocity], SetFactor[Position]]:
    # Mountaincar stops if it reaches the edge
    shape = obj.Shape.value
    v = obj[Velocity]
    x = obj[Position]
    b = wall.boundaries

    if wall.damping:
        v_ = v * (1 - wall.damping)
    else:
        v_ = v

    if isinstance(shape, Circle):
        # find hypotenuse of the right angle defined by the unit
        # vector and overlap, p, with wall.
        r = shape.radius

        dist_top = x[1] + r - b[1]
        dist_bottom = b[0] - x[1] + r
        dist_left = b[0] - x[0] + r
        dist_right = x[0] + r - b[1]

        min_rl = min(-dist_right, -dist_left)
        min_tb = min(-dist_top, -dist_bottom)

        if min_rl < min_tb:
            vx = 0.
            vy = v_[1]
        elif min_rl > min_tb:
            vx = v_[0]
            vy = -v_[1]
        else:
            vx = 0.
            vy = -v_[1]

        vf = [vx, vy]

    else:
        raise NotImplementedError(shape)

    return SetFactor[Velocity](v, vf)

@TransitionFunction
def change_angle(theta: Angle, v: AngularVelocity) -> Differential[Angle]:
    dtheta = v
    return Differential[Angle](theta, dtheta)


@TransitionFunction
def change_angular_velocity(thetavel: AngularVelocity, thetaacc: AngularAcceleration) -> Differential[AngularVelocity]:
    dthetavel = thetaacc
    return Differential[AngularVelocity](thetavel, thetaacc)


@TransitionFunction
def change_angular_acceleration(o_factors: Tuple[Mass, Velocity, PoleMassProp, PoleLength, Acceleration, Angle, AngularVelocity, AngularAcceleration], gravity: Gravity
                               ) -> Tuple[Aggregate[AngularAcceleration], Aggregate[Acceleration]]:
    total_mass, vel, pole_prop_mass, length, acceleration, theta, thetavel, thetaacc = o_factors

    if abs(theta) >= np.pi / 2.:
        # This is pretty much game over for carpole, but if we want cartpole physics that work past when the episode normally ends, we need to stop theta
        return None
    a_y = acceleration[1]
    pole_mass = total_mass * pole_prop_mass
    force = a_y * total_mass
    sintheta = np.sin(theta)
    costheta = np.cos(theta)
    
    # Copied more or less from gym
    temp = (force + length * thetavel ** 2 * sintheta) / total_mass
    dthetavel = (gravity * sintheta - costheta * temp) / (length * (4.0 / 3.0 - pole_mass * costheta ** 2 / total_mass))

    # Here we need to remove the external force because it's already being applied to the ball
    da_y = temp - length * dthetavel * costheta / total_mass - force / total_mass
    
    return Aggregate[AngularAcceleration](thetaacc, dthetavel), Aggregate[Acceleration](acceleration, [0., da_y])


def color_map(thing: Thing):
    if not thing.has_factor(Angle):
        return None
    
    theta = thing[Angle]
    c = round((theta + float(np.pi / 2.)) * 255 / float(np.pi))
    color = [c, 0, 255 - c]
    return color


register_rule(color_map)


class CartPole(Ball, default={Shape: Circle(0.2), Mobile: True, Label: "cartpole", Text: "C"}):
    _factor_types = Object._factor_types + (Angle, AngularVelocity, AngularAcceleration, PoleMassProp, PoleLength)

sim.add_rule(change_angle)
sim.add_rule(change_angular_velocity)
sim.add_rule(change_angular_acceleration)
sim.add_rule(accelerate_like_mountian_car)
sim._wall_collision_rule = mountaincar_wall_collision

In [ ]:
_X_ACTIONS = [-2., 0, 2.]
_Y_ACTIONS = [-1., 1.]


class MountainCartPoleInitialization(Initialization):

    def __call__(self, init_things: Optional[list[Entity]] = None) -> None:
        cartpole = CartPole(initial_factors={Angle: 0.01})
        sim.adopt(cartpole)

        
class MountainCartPoleTask(Task):

    def __init__(
        self,
        initialization: Initialization,
        x_actions: list[float] = _X_ACTIONS,
        y_actions: list[float] = _Y_ACTIONS,
        baseline_action: int = 0
    ):

        self._sim = None
        actions = []
        for x_action in x_actions:
            for y_action in y_actions:
                actions.append([x_action, y_action])
        self._actions = actions
        self._action_space = Discrete(len(actions))
        self._initialization = initialization
        self._baseline_action = baseline_action

    @property
    def action_space(self) -> Discrete:
        return self._action_space

    def check_action(self, action: int) -> bool:
        return action in list(range(len(self._actions)))

    def demo_action(self) -> int:
        return 4 # Do nothing mountaincar

    def apply_baseline(self, action: int) -> int:
        self.check_action(action)
        action += self._baseline_action
        return action

    def reward(self, state: dict) -> float:
        size = state['things'][0][Size]
        pos_x = state['things'][0][Position][0]
        pos_x = change_basis(pos_x)
        if pos_x >= 0.4:
            reward = 1.0
        else:
            reward = -0.01
            
        theta = state['things'][0][Angle]
        return float(np.abs(theta) <= np.pi / 2.) + reward

    def apply_action(self, action: int) -> None:
        self.sim.add_force(0, np.array(self._actions[action]))

    def done(self, state: dict) -> bool:
        size = state['things'][0][Size]
        pos_x = state['things'][0][Position][0]
        pos_x = change_basis(pos_x)
        return pos_x >= 0.4

    def sample(self) -> None:
        pass

    def results(self, state: dict) -> dict:
        """Returns results to be processed by the MDP.
        :param state: State dictionary to pull results from.
        :return: Dictionary of results.
        """
        return {}